<a href="https://colab.research.google.com/github/alikaiser12/AI/blob/main/LangChain_Custom_Tool_Integration_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# ===============================================
# Single-Cell Colab Demo (No pydantic error, fully working as of Oct 2024)
# Uses modern package structure: langchain-core, langchain-community, and langchain
# and imports FakeListLLM from langchain_core.language_models.fake.
# ===============================================

# 1) Install compatible versions
!pip install -q -U "pydantic>=2.7,<3" \
                  "langchain>=0.3.0,<0.4" \
                  "langchain-core>=0.3.0,<0.4" \
                  "langchain-community>=0.3.0,<0.4"

# 2) Imports (note the langchain_core paths)
from langchain_core.tools import Tool
from langchain_core.language_models.fake import FakeListLLM
from langchain.agents import initialize_agent, AgentType

# 3) Mock API function
def query_order_status(order_id: str) -> str:
    """Return a fake order status for the given order_id."""
    return "Shipped" if order_id == "123" else "In Process"

# 4) Create the custom tool via Tool (not StructuredTool)
order_checker_tool = Tool(
    name="OrderCheckerTool",
    func=query_order_status,
    description=(
        "Check the status of an order by ID. "
        "Use this tool when the user asks for an order status and provides the order_id."
    ),
)

# 5) Simulated LLM responses
fake_responses = [
    "I'll check the first order via OrderCheckerTool.\nAction: OrderCheckerTool\nAction Input: 123",
    "Got the first result. Now I'll check the second order.\nAction: OrderCheckerTool\nAction Input: 999",
    "Final Answer: Order 123 is Shipped and Order 999 is In Process."
]
llm = FakeListLLM(responses=fake_responses)

# 6) Initialize agent and run test query
agent = initialize_agent(
    tools=[order_checker_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "I have two order IDs: '123' and '999'. What is the status of each of these orders?"
print("User Query:", query)
result = agent.run(query)

# 7) Final, client-facing confirmation
print("\n" + "="*60)
print("Final Answer:")
print(result)
print("="*60)

User Query: I have two order IDs: '123' and '999'. What is the status of each of these orders?


> Entering new AgentExecutor chain...
I'll check the first order via OrderCheckerTool.
Action: OrderCheckerTool
Action Input: 123
Observation: Shipped
Thought:Got the first result. Now I'll check the second order.
Action: OrderCheckerTool
Action Input: 999
Observation: In Process
Thought:Final Answer: Order 123 is Shipped and Order 999 is In Process.

> Finished chain.

Final Answer:
Order 123 is Shipped and Order 999 is In Process.


/tmp/ipython-input-2061919265.py:39: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(
/tmp/ipython-input-2061919265.py:48: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(query)
